In [1]:
import os 
import datetime
from pathlib import Path
from dotenv import load_dotenv, find_dotenv

basepath = Path(os.getcwd())
# make sure your working directory is the repository root.
if basepath.name != "idp-radio-1":
    os.chdir(basepath.parent.parent.parent)
load_dotenv(find_dotenv())

%load_ext autoreload
%autoreload 2
os.getcwd()

'/srv/idp-radio-1'

In [2]:
import os 
import tensorflow as tf
from pathlib import Path

In [3]:
# Run this before loading other dependencies, otherwise they might occupy memory on gpu 0 by default and it will stay that way

# Specify which GPU(s) to use
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # Or 2, 3, etc. other than 0

config = tf.compat.v1.ConfigProto(allow_soft_placement=True, log_device_placement=True)
config.gpu_options.allow_growth = True
tf.compat.v1.Session(config=config)


Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce RTX 2080 Ti, pci bus id: 0000:08:00.0, compute capability: 7.5



In [4]:

from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.densenet import DenseNet121
from tensorflow.keras.applications.resnet_v2 import ResNet101V2
from src.architectures.simple.simple_base import SimpleBaseArchitecture
from src.architectures.adv.guendel19 import densenet

from src.architectures.benchmarks.benchmark import Benchmark, Experiment
from src.architectures.benchmarks.benchmark_definitions import Chexpert_Benchmark, Chestxray14_Benchmark, CHEXPERT_BENCHMARKS, CHESTXRAY14_BENCHMARKS, METRICS, CHEXPERT_COLUMNS, CHESTXRAY14_COLUMNS
from src.metrics.metrics import F2Score
from src.metrics.losses import WeightedBinaryCrossentropy, compute_class_weight

Using TensorFlow backend.


In [5]:
CHEXPERT_BENCHMARKS.keys() 

dict_keys(['BCE_E3_B16', 'WBCE_E3_B16', 'BCE_E10_B16', 'WBCE_E10_B16', 'BCE_E20_B16', 'WBCE_E20_B16', 'BCE_E50_B16', 'WBCE_E50_B16', 'BCE_E100_B16', 'WBCE_E100_B16', 'BCE_E3_B32', 'WBCE_E3_B32', 'BCE_E10_B32', 'WBCE_E10_B32', 'BCE_E20_B32', 'WBCE_E20_B32', 'BCE_E50_B32', 'WBCE_E50_B32', 'BCE_E100_B32', 'WBCE_E100_B32', 'BCE_E3_B64', 'WBCE_E3_B64', 'BCE_E10_B64', 'WBCE_E10_B64', 'BCE_E20_B64', 'WBCE_E20_B64', 'BCE_E50_B64', 'WBCE_E50_B64', 'BCE_E100_B64', 'WBCE_E100_B64'])

In [6]:
#CHESTXRAY14_BENCHMARKS.keys()

In [7]:
#[m.name for m in METRICS]

In [8]:
#CHEXPERT_COLUMNS

In [9]:
#CHESTXRAY14_COLUMNS

In [10]:
!remote_access/get_tunnels.sh

URLs open for the following services:
[('tensorboard', 'http://d96140c9e40a.ngrok.io'), ('jupyterlab', 'http://ce3962148fb2.ngrok.io')]


In [11]:
def simple_architecture_experiment(benchmark, base_model_fn, classes ):
    model = SimpleBaseArchitecture(base_model_fn, len(classes))
    experiment = Experiment(benchmark, model)
    return experiment

In [15]:
chexpert_resnet_exp = simple_architecture_experiment(CHEXPERT_BENCHMARKS["WBCE_E20_B32"], DenseNet121, CHEXPERT_COLUMNS)


In [13]:
print(chexpert_resnet_exp.model_description )


Trained DenseNet121 architecture using the 'Chexpert Weighted BCE Quick Training Epochs and Medium Batches' benchmark. The benchmark was initialized for the chexpert_full dataset with batch size of 32, shuffel set to True and images rescaled to dimension (256, 256).
The training was done for 20 epochs using the Adam optimizer and weighted_binary_crossentropy loss.
A total of 14 labels/pathologies were included in the training and encoded using the 'uzeroes' method.
The traing set included 141807 number of sample, the validation set 36980, and the test set 44627. 


In [16]:
chexpert_resnet_exp.run()

Epoch 1/20
   1/4431 [..............................] - ETA: 0s - loss: 1.4407 - auc: 0.4264 - precision: 0.1856 - recall: 0.7101 - f2_score: 0.4537 - binary_accuracy: 0.4754WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
4431/4431 [==============================] - ETA: 0s - loss: 1.2495 - auc: 0.6897 - precision: 0.2615 - recall: 0.7096 - f2_score: 0.5285 - binary_accuracy: 0.6223
Epoch 00001: saving model to models/DenseNet1212020-06-08-22-50-50/weights.01-1.44.hdf5
4431/4431 [==============================] - 943s 213ms/step - loss: 1.2495 - auc: 0.6897 - precision: 0.2615 - recall: 0.7096 - f2_score: 0.5285 - binary_accuracy: 0.6223 - val_loss: 1.4400 - val_auc: 0.6943 - val_precision: 0.2268 - val_recall: 0.3658 - val_f2_score: 0.3258 - val_binary_accura

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



4431/4431 [==============================] - ETA: 0s - loss: 1.1603 - auc: 0.7490 - precision: 0.2996 - recall: 0.7474 - f2_score: 0.5754 - binary_accuracy: 0.6708
Epoch 00003: saving model to models/DenseNet1212020-06-08-22-50-50/weights.03-1.21.hdf5
4431/4431 [==============================] - 940s 212ms/step - loss: 1.1603 - auc: 0.7490 - precision: 0.2996 - recall: 0.7474 - f2_score: 0.5754 - binary_accuracy: 0.6708 - val_loss: 1.2050 - val_auc: 0.7425 - val_precision: 0.2640 - val_recall: 0.6922 - val_f2_score: 0.5227 - val_binary_accuracy: 0.6277
Epoch 4/20
1584/4431 [=========>....................] - ETA: 8:47 - loss: 1.1411 - auc: 0.7582 - precision: 0.3070 - recall: 0.7524 - f2_score: 0.5832 - binary_accuracy: 0.6802

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



3539/4431 [======================>.......] - ETA: 2:45 - loss: 1.1091 - auc: 0.7758 - precision: 0.3182 - recall: 0.7623 - f2_score: 0.5960 - binary_accuracy: 0.6919

...

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



41099/44627 [==========================>...] - ETA: 46s

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



34761/44627 [======================>.......] - ETA: 2:46 - loss: 1.3386 - auc: 0.7704 - precision: 0.3711 - recall: 0.7458 - f2_score: 0.6205 - binary_accuracy: 0.7503

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [17]:
chestxray14_resnet_exp = simple_architecture_experiment(CHESTXRAY14_BENCHMARKS["WBCE_E20_B32"], DenseNet121, CHESTXRAY14_COLUMNS)

In [18]:
print(chestxray14_resnet_exp.model_description )


Trained DenseNet121 architecture using the 'Chestxray NIH 14 Weighted BCE Quick Training Epochs and Medium Batches' benchmark. The benchmark was initialized for the chestxray14_scale_256 dataset with batch size of 32, shuffel set to True and images rescaled to dimension (256, 256).
The training was done for 20 epochs using the Adam optimizer and weighted_binary_crossentropy loss.
A total of 15 labels/pathologies were included in the training and encoded using the 'uzeroes' method.
The traing set included 71310 number of sample, the validation set 17718, and the test set 23092. 


In [19]:
chestxray14_resnet_exp.run()

Epoch 1/20


ValueError: in user code:

    /usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:803 train_function  *
        return step_function(self, iterator)
    /usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:793 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /usr/local/lib/python3.6/dist-packages/tensorflow/python/distribute/distribute_lib.py:969 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /usr/local/lib/python3.6/dist-packages/tensorflow/python/distribute/distribute_lib.py:2352 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /usr/local/lib/python3.6/dist-packages/tensorflow/python/distribute/distribute_lib.py:2711 _call_for_each_replica
        return fn(*args, **kwargs)
    /usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:786 run_step  **
        outputs = model.tr

{'benchmark_name': 'Chestxray NIH 14 Weighted BCE Dev Epochs and Medium Batches',
 'dataset_name': 'chestxray14_scale_256',
 'dataset_folder': 'data/chestxray14/scale_256',
 'models_dir': 'models',
 'epochs': 10,
 'optimizer': 'Adam',
 'loss': 'weighted_binary_crossentropy',
 'metrics': ['auc', 'precision', 'recall', 'f2_score', 'binary_accuracy'],
 'label_columns': ['Edema',
  'Atelectasis',
  'Pneumonia',
  'Pleural_Thickening',
  'Cardiomegaly',
  'Infiltration',
  'Consolidation',
  'Fibrosis',
  'No Finding',
  'Effusion',
  'Nodule',
  'Mass',
  'Hernia',
  'Emphysema',
  'Pneumothorax'],
 'path_column': 'Image Index',
 'path_column_prefix': 'images/',
 'shuffle': True,
 'batch_size': 32,
 'dim': (256, 256),
 'n_channels': 3,
 'nan_replacement': 0,
 'unc_value': -1,
 'u_enc': 'uzeroes',
 'drop_last': True,
 'train_num_samples': 71779,
 'valid_num_samples': 17703,
 'test_num_samples': 22638}

In [ ]:
predictions = experiment_inceptionnet_chexpert.model.predict(
            experiment_inceptionnet_chexpert.benchmark.testgen, steps=len(experiment_inceptionnet_chexpert.benchmark.testgen), verbose=1)

In [ ]:
true_labels = experiment_inceptionnet_chexpert.benchmark.testgen.get_encoded_labels()

In [14]:
# model_guendel_chestxray14 = densenet(classes=len(CHESTXRAY14_COLUMNS))
model_guendel_chexpert = densenet(classes=len(CHEXPERT_COLUMNS))

#experiment_guendel_chestxray14 = Experiment(chestxray14_benchmark, model_guendel_chestxray14)
experiment_guendel_chexpert = Experiment(CHEXPERT_BENCHMARKS["WBCE_E10_B32"], model_guendel_chexpert, model_name="test_WBCE_32")



In [15]:
#experiment_guendel_chestxray14_result =  experiment_guendel_chestxray14.run()

In [16]:
experiment_guendel_chexpert_result =  experiment_guendel_chexpert.run()

Epoch 1/10
 584/4497 [==>...........................] - ETA: 52:37 - loss: 1.3441

KeyboardInterrupt: 

In [14]:
experiment_guendel_chexpert.evaluate()

    5/44370 [..............................] - ETA: 9:24 - loss: 1.0810 - auc: 0.3036 - precision: 0.3846 - recall: 0.3846 - f2_score: 0.3846 - binary_accuracy: 0.7714   

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


44370/44370 [==============================] - 717s 16ms/step - loss: 1.3040 - auc: 0.7536 - precision: 0.3530 - recall: 0.6325 - f2_score: 0.5460 - binary_accuracy: 0.7490


{'report': '                            precision    recall  f1-score   support\n\n                No Finding       0.10      0.38      0.15      4383\nEnlarged Cardiomediastinum       0.05      0.46      0.09      2137\n              Cardiomegaly       0.12      0.33      0.18      5454\n              Lung Opacity       0.47      0.40      0.43     21069\n               Lung Lesion       0.04      0.22      0.07      1931\n                     Edema       0.24      0.24      0.24     10439\n             Consolidation       0.07      0.25      0.11      3030\n                 Pneumonia       0.03      0.11      0.05      1256\n               Atelectasis       0.14      0.37      0.21      6421\n              Pneumothorax       0.09      0.20      0.12      3816\n          Pleural Effusion       0.37      0.40      0.39     16713\n             Pleural Other       0.02      0.17      0.03       745\n                  Fracture       0.04      0.17      0.06      1726\n           Support D

In [19]:
experiment_guendel_chexpert.save(upload=False)

'9540a918-a699-11ea-a7d0-0242ac110005'

In [20]:
print("done")

done


In [18]:
CHEXPERT_BENCHMARKS["WBCE_E10_B32"].as_dict()

{'benchmark_name': 'Chexpert Weighted BCE Dev Epochs and Medium Batches',
 'dataset_name': 'chexpert_full',
 'dataset_folder': 'data/chexpert/full',
 'models_dir': 'models',
 'epochs': 10,
 'optimizer': 'Adam',
 'loss': 'weighted_binary_crossentropy',
 'metrics': ['AUC', 'Precision', 'Recall', 'F2Score', 'BinaryAccuracy'],
 'label_columns': ['No Finding',
  'Enlarged Cardiomediastinum',
  'Cardiomegaly',
  'Lung Opacity',
  'Lung Lesion',
  'Edema',
  'Consolidation',
  'Pneumonia',
  'Atelectasis',
  'Pneumothorax',
  'Pleural Effusion',
  'Pleural Other',
  'Fracture',
  'Support Devices'],
 'path_column': 'Path',
 'path_column_prefix': '',
 'shuffle': True,
 'batch_size': 32,
 'dim': (256, 256),
 'n_channels': 3,
 'nan_replacement': 0,
 'unc_value': -1,
 'u_enc': 'uzeroes',
 'drop_last': True,
 'train_num_samples': 143398,
 'valid_num_samples': 35646,
 'test_num_samples': 44370}